# Определение текущего уровня потребительской лояльности

**Заказчик исследования** — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ.
**Цель** - определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России.
Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».

Для достижения поставленной цели решим следующие **задачи**:
1. Подключение к базе

2. Выгрузка данных
Подготовка данных для дашборда с использованием SQL-запроса, который выгрузит следующие поля:

- user_id - идентификатор клиента
- lt_day - количество дней «жизни» клиента
- is_new - поле хранит информацию о том, является ли клиент новым
- age - возраст
- gender_segment - пол (для удобства работы с полем преобразовать значения в текстовый вид)
- os_name - тип операционной системы
- cpe_type_name - тип устройства
- country - страна проживания
- city - город проживания
- age_segment - возрастной сегмент
- traffic_segment - сегмент по объёму потребляемого трафика
- lifetime_segment - сегмент по количеству дней «жизни»
- nps_score - оценка клиента в NPS-опросе
- nps_group - поле хранит информацию о том, к какой группе относится оценка клиента в опросе

3. Создание дашбордов в Tableau
4. Создание презентации

**Описание данных**

Таблица user:
- user_id - идентификатор клиента, первичный ключ таблицы
- lt_day - количество дней «жизни» клиента
- age - возраст клиента в годах
- gender_segment - пол клиента (1 – женщина, 0 – мужчина)
- os_name - тип операционной системы
- cpe_type_name - тип устройства
- location_id - идентификатор домашнего региона клиента, внешний ключ, отсылающий к таблице location
- age_gr_id - идентификатор возрастного сегмента клиента, внешний ключ, отсылающий к таблице age_segment
- tr_gr_id - идентификатор сегмента клиента по объёму потребляемого трафика в месяц, внешний ключ, отсылающий к таблице traffic_segment
- lt_gr_id - идентификатор сегмента клиента по количеству месяцев «жизни», внешний ключ, отсылающий к таблице lifetime_segment
- nps_score - оценка клиента в NPS-опросе (от 1 до 10)

Таблица location:
- location_id - идентификатор записи, первичный ключ
- country - страна
- city -город

Таблица age_segment:
- age_gr_id - идентификатор сегмента, первичный ключ
- bucket_min - минимальная граница сегмента
- bucket_max - максимальная граница сегмента
- title - название сегмента

Таблица traffic_segment:
- tr_gr_id - идентификатор сегмента, первичный ключ
- bucket_min - минимальная граница сегмента
- bucket_max - максимальная граница сегмента
- title - название сегмента

Таблица lifetime_segment:
- lt_gr_id - идентификатор сегмента, первичный ключ
- bucket_min - минимальная граница сегмента
- bucket_max - максимальная граница сегмента
- title - название сегмента


In [ ]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [ ]:
#path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

#if os.path.exists(path_to_db_local):
#    path_to_db = path_to_db_local
if os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

**Посмотрим поближе на таблицы:**

In [ ]:
query = """
        SELECT *
        FROM user
        LIMIT 15;
        """
user = pd.read_sql(query, engine)
user

,user_id,lt_day,age,gender_segment,os_name,cpe_type_name,location_id,age_gr_id,tr_gr_id,lt_gr_id,nps_score
0,A001A2,2320,45.0,1.0,ANDROID,SMARTPHONE,55,5,5,8,10
1,A001WF,2344,53.0,0.0,ANDROID,SMARTPHONE,21,5,5,8,10
2,A003Q7,467,57.0,0.0,ANDROID,SMARTPHONE,28,6,9,6,10
3,A004TB,4190,44.0,1.0,IOS,SMARTPHONE,38,4,4,8,10
4,A004XT,1163,24.0,0.0,ANDROID,SMARTPHONE,39,2,6,8,10
5,A005O0,5501,42.0,1.0,ANDROID,SMARTPHONE,34,4,6,8,6
6,A0061R,1236,45.0,0.0,ANDROID,SMARTPHONE,55,5,7,8,10
7,A009KS,313,35.0,0.0,ANDROID,SMARTPHONE,28,4,14,5,10
8,A00AES,3238,36.0,1.0,ANDROID,SMARTPHONE,41,4,5,8,10
9,A00F70,4479,54.0,1.0,ANDROID,SMARTPHONE,9,5,8,8,9


**Рассмотрим некоторые столбцы поближе**

In [ ]:
query = """
        SELECT DISTINCT lt_day
        FROM user
        ORDER BY lt_day;
        """
u_dab_lt = pd.read_sql(query, engine)
u_dab_lt

,lt_day
0,-21
1,-13
2,-12
3,-11
4,-8
...,...
6945,8385
6946,8490
6947,8640
6948,8828


**Вряд ли количество дней "жизни" клиента может иметь отрицательную величину. Посчитаем, сколько строк с отрицательным значением:**

In [ ]:
query = """
        SELECT COUNT(*)
        FROM user
        WHERE lt_day <0
        """
u_lt_otr = pd.read_sql(query, engine)
u_lt_otr

,COUNT(*)
0,13


In [ ]:
query = """
        SELECT COUNT(*)
        FROM user
        """
u_stroki = pd.read_sql(query, engine)
u_stroki

,COUNT(*)
0,502493


**Таких строк мало, всего 0,003%**

In [ ]:
query = """
        SELECT DISTINCT age
        FROM user
        ORDER BY age;
        """
u_dab_age = pd.read_sql(query, engine)
u_dab_age

,age
0,NaN
1,10.0
2,11.0
3,12.0
4,13.0
...,...
75,84.0
76,85.0
77,86.0
78,87.0


In [ ]:
query = """
        SELECT COUNT(*)
        FROM user
        WHERE age is NULL
        """
u_age_null = pd.read_sql(query, engine)
u_age_null

,COUNT(*)
0,554


**Пропущенных значений в этом столбце 0,11%**

In [ ]:
query = """
        SELECT DISTINCT gender_segment
        FROM user;
        """
u_dab_gen = pd.read_sql(query, engine)
u_dab_gen

,gender_segment
0,1.0
1,0.0
2,NaN


In [ ]:
query = """
        SELECT COUNT(*)
        FROM user
        WHERE gender_segment is NULL
        """
u_gen_null = pd.read_sql(query, engine)
u_gen_null

,COUNT(*)
0,1301


**Пропущенных значений в этом столбце 0,26%**

In [ ]:
query = """
        SELECT DISTINCT nps_score
        FROM user
        ORDER BY nps_score;
        """
u_dab_nps = pd.read_sql(query, engine)
u_dab_nps

,nps_score
0,1
1,2
2,3
3,4
4,5
5,6
6,7
7,8
8,9
9,10


In [ ]:
query = """
        SELECT *
        FROM location
        LIMIT 15;
        """
location = pd.read_sql(query, engine)
location

,location_id,city,country
0,1,Архангельск,Россия
1,2,Астрахань,Россия
2,3,Балашиха,Россия
3,4,Барнаул,Россия
4,5,Белгород,Россия
5,6,Брянск,Россия
6,7,Владивосток,Россия
7,8,Владимир,Россия
8,9,Волгоград,Россия
9,10,Волжский,Россия


**Посмотрим на столбцы поближе**

In [ ]:
query = """
        SELECT DISTINCT city
        FROM location
        ORDER BY city;
        """
loc_dab_city = pd.read_sql(query, engine)
loc_dab_city

,city
0,Архангельск
1,Астрахань
2,Балашиха
3,Барнаул
4,Белгород
...,...
57,Челябинск
58,Череповец
59,Чита
60,Якутск


In [ ]:
query = """
        SELECT DISTINCT country
        FROM location;
        """
loc_dab_coun = pd.read_sql(query, engine)
loc_dab_coun

,country
0,Россия


In [ ]:
query = """
        SELECT *
        FROM age_segment
        LIMIT 15;
        """
age_segment = pd.read_sql(query, engine)
age_segment

,age_gr_id,bucket_min,bucket_max,title
0,1,NaN,15.0,01 до 16
1,2,16.0,24.0,02 16-24
2,3,25.0,34.0,03 25-34
3,4,35.0,44.0,04 35-44
4,5,45.0,54.0,05 45-54
5,6,55.0,64.0,06 55-64
6,7,66.0,NaN,07 66 +
7,8,NaN,NaN,08 n/a


In [ ]:
query = """
        SELECT *
        FROM traffic_segment
        LIMIT 15;
        """
traffic_segment = pd.read_sql(query, engine)
traffic_segment

,tr_gr_id,bucket_min,bucket_max,title
0,1,0.00,0.00,01 0
1,2,0.00,0.01,01 0-0.01
2,3,0.01,0.10,02 0.01-0.1
3,4,0.10,1.00,03 0.1-1
4,5,1.00,5.00,04 1-5
5,6,5.00,10.00,05 5-10
6,7,10.00,15.00,06 10-15
7,8,15.00,20.00,07 15-20
8,9,20.00,25.00,08 20-25
9,10,25.00,30.00,09 25-30


In [ ]:
query = """
        SELECT *
        FROM lifetime_segment
        LIMIT 15;
        """
lifetime_segment = pd.read_sql(query, engine)
lifetime_segment

,lt_gr_id,bucket_min,bucket_max,title
0,1,1.0,1.0,01 1
1,2,2.0,2.0,02 2
2,3,3.0,3.0,03 3
3,4,4.0,6.0,04 4-6
4,5,7.0,12.0,05 7-12
5,6,13.0,24.0,06 13-24
6,7,25.0,36.0,07 25-36
7,8,36.0,NaN,08 36+


**Для формирования нужной таблицы составим SQL-запрос. Срузу избавимся от пропущенных значений и аномалий - потеря данных будет не велика, не больше 0,5%.**

In [ ]:
query = '''
        SELECT user_id,
               lt_day,
               CASE
                    WHEN lt_day < 365 THEN 'Новый'
                    WHEN lt_day >= 365 THEN 'Старый'
                    END AS is_new,
               age,
               CASE
                    WHEN gender_segment = 0 THEN 'Мужчина'
                    WHEN gender_segment = 1 THEN 'Женщина'
                    ELSE NULL
                    END AS gender_segment,
               os_name,
               cpe_type_name,
               l.country,
               l.city,
               SUBSTRING(a.title, 3) AS age_segment,
               SUBSTRING(t.title, 3) AS traffic_segment,
               SUBSTRING(lt.title, 3) AS lifetime_segment,
               nps_score,
               CASE
                    WHEN nps_score >= 9 THEN 'Сторонники'
                    WHEN nps_score > 6 AND nps_score < 9 THEN 'Нейтралы'
                    WHEN nps_score <= 6 THEN 'Критики'
               END AS nps_group
         FROM user AS u
         INNER JOIN (
                SELECT location_id,
                CASE
                    WHEN city = 'СанктПетербург' THEN 'Санкт-Петербург'
                    WHEN city = 'НижнийНовгород' THEN 'Нижний Новгород'
                    WHEN city = 'РостовнаДону' THEN 'Ростов-на-Дону'
                    WHEN city = 'НабережныеЧелны' THEN 'Набережные Челны'
                    WHEN city = 'УланУдэ' THEN 'Улан-Удэ'
                    WHEN city = 'НижнийТагил' THEN 'Нижний Тагил'
                    ELSE city END AS city,
                country
                FROM location
            ) AS l ON u.location_id=l.location_id
         INNER JOIN age_segment AS a ON u.age_gr_id = a.age_gr_id
         INNER JOIN traffic_segment AS t ON u.tr_gr_id = t.tr_gr_id
         INNER JOIN lifetime_segment AS lt ON u.lt_gr_id = lt.lt_gr_id
         WHERE lt_day >=0 AND age IS NOT NULL AND gender_segment IS NOT NULL;
         '''

**Поправила назания городов**

In [ ]:
df = pd.read_sql(query, engine)
df.head(15)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,Старый,45.0,Женщина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,Сторонники
1,A001WF,2344,Старый,53.0,Мужчина,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,Сторонники
2,A003Q7,467,Старый,57.0,Мужчина,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,Сторонники
3,A004TB,4190,Старый,44.0,Женщина,IOS,SMARTPHONE,Россия,Ростов-на-Дону,35-44,0.1-1,36+,10,Сторонники
4,A004XT,1163,Старый,24.0,Мужчина,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,Сторонники
5,A005O0,5501,Старый,42.0,Женщина,ANDROID,SMARTPHONE,Россия,Омск,35-44,5-10,36+,6,Критики
6,A0061R,1236,Старый,45.0,Мужчина,ANDROID,SMARTPHONE,Россия,Уфа,45-54,10-15,36+,10,Сторонники
7,A009KS,313,Новый,35.0,Мужчина,ANDROID,SMARTPHONE,Россия,Москва,35-44,45-50,7-12,10,Сторонники
8,A00AES,3238,Старый,36.0,Женщина,ANDROID,SMARTPHONE,Россия,Санкт-Петербург,35-44,1-5,36+,10,Сторонники
9,A00F70,4479,Старый,54.0,Женщина,ANDROID,SMARTPHONE,Россия,Волгоград,45-54,15-20,36+,9,Сторонники


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501152 entries, 0 to 501151
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           501152 non-null  object 
 1   lt_day            501152 non-null  int64  
 2   is_new            501152 non-null  object 
 3   age               501152 non-null  float64
 4   gender_segment    501152 non-null  object 
 5   os_name           501152 non-null  object 
 6   cpe_type_name     501152 non-null  object 
 7   country           501152 non-null  object 
 8   city              501152 non-null  object 
 9   age_segment       501152 non-null  object 
 10  traffic_segment   501152 non-null  object 
 11  lifetime_segment  501152 non-null  object 
 12  nps_score         501152 non-null  int64  
 13  nps_group         501152 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 53.5+ MB


In [ ]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

**Ссылка на презентацию:**
https://public.tableau.com/views/2_17341063813600/sheet26?:language=en-US&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link

**Выводы и наблюдения:**

Дашборд "Участники опроса"
Выводы:
1. Больше в опросе участвовало женщин.
2. Больше в опросе участвовало клиентов в возрасте от 35 до 44 лет.
3. Так же больше в опросе участвовало старых клиентов, с количеством дней "жизни" более 365.
4. Основная часть опрошенных проживает в Москве.

Дашборд "Лояльность клиентов"
Выводы:
1. Количество лояльных клиентов (сторонников) превышает критиков и нейтралов.
2. Более лояльны "старые" клиенты (с "жизнью" более 36 месяцев) женского пола в возрасте 35-44 лет, которые проживают в Москве и используют SMARTHONE на базе ANDROID с количеством потребляемого трафика от 1 до 5.
3. Менее лояльны - "новые" клиенты (с "жизнью" 3 месяца) мужского пола в возрасте до 16 лет, которые проживают в Череповецке и используют MIFI ROUTER на базе WINDOWS MOBILE с количеством потребляемого трафика от 95 до 100.

Дашборд "Анализ NPS"
Выводы:
1. Общий NPS составил 21,97%.
2. Клиентов, которые относятся к группе сторонников можно описать как "нового" пользователя (с "жизнью" 1 месяц) женского пола старше 66 лет, проживающего в городе Иваново, использующего MIFI ROUTER и ОС - SYMBAIN OS с количеством потребляемого трафика 0-0,01.

P.S.: "жизнь" - сколько времени клиент находится на обслуживании.